In [14]:
import time, os
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert

import warnings
warnings.filterwarnings(action='ignore')

위키 백화의 장르 설명에서 키워드를 추출하여 저장

### 위키백과 크롤링

In [15]:
# 드라이버 지정
current = os.getcwd()
chrome_path = current + '/driver/chromedriver' 

options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
# options.add_argument('headless') # 창 안 띄움 -> 더 빠름
# options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") # Headless 탐지 막기위해 User-Agent 값을 바꿔줌.

options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')

In [26]:
genre_script = []
genre_list = ['액션', '코미디', '다큐멘터리', '판타지', '공포', '음악', '로맨스', '스포츠', '서부', 'Made in Europe', '애니메이션', '범죄', '드라마', '역사', '가족', '스릴러', 'SF', '전쟁', 'Reality TV']

driver = webdriver.Chrome(chrome_path, options=options)

for genre in genre_list:
    url = "https://ko.wikipedia.org/wiki/{}_영화".format(genre)
    driver.get(url)
    soup = BeautifulSoup(url)
    genre_script.append(driver.find_element_by_css_selector(".vector-body p").text)

driver.close()

In [39]:
data = {'genre':genre_list, 'script':genre_script}
genre_df = pd.DataFrame(data)

In [42]:
genre_df.head(2)

,genre,script
0,액션,"액션 영화(action film)는 영화의 한 갈래로, 등장인물의 육체적인 움직임을..."
1,코미디,"코미디 영화(comedy film)는 유머에 중점을 둔 영화 장르로, 관객들로부터 ..."
2,다큐멘터리,"다큐멘터리 영화(영어: Documentary Film), 또는 기록 영화(記錄映畵,..."
3,판타지,판타지 영화란 판타지적인 내용을 영화로 만든 것이다.
4,공포,"공포 영화(恐怖映畫, 영어: horror film)란 귀신이나 유령 등 무서운 것과..."
5,음악,"뮤지컬 영화(Musical film)는 영화의 장르 중 하나로, 이야기에 혼재하여 ..."
6,로맨스,"로맨스 영화(영어: romance film)는 영화의 장르 중 하나로, 사랑을 주제..."
7,스포츠,"스포츠 영화(-映畵)는 스포츠를 소재로 한 영화로, 스포츠 선수나 팀, 경기 등을 ..."
8,서부,서부극(西部劇)은 미국의 역사 중에서 서부 개척사를 소재로 한 영화나 극 작품이다.
9,Made in Europe,위키백과에 ‘Made in Europe 영화’ 문서가 존재하지 않습니다.


In [43]:
genre_df[genre_df['genre']=='Reality TV'].script = "리얼리티 방송(영어: reality television)은 대본에 기반하지 않는, 멜로드라마틱한 상황이나 유머스러운 상황을 표출하는 텔레비전 프로그램의 한 장르로, 전문 배우 대신 일반인을 주연으로 하는 것이 보통이며 상을 수여하는 콘테스트에서도 이따금 이루어진다.[1] 리얼리티 방송은 1948년 TV 시리즈 캔디드 카메라(Candid Camera)에서 비롯한다.[2] 이 장르는 1999년부터 2000년 즈음 빅 브라더와 서바이벌과 같은 텔레비전 시리즈의 성공과 더불어 하나의 현상으로 급격히 발전하였다.[1] 리얼리티 방송 장르에 속하는 프로그램을 흔히 리얼리티 쇼(reality shows)라 부르며 텔레비전 시리즈로 자주 제작된다. 다큐멘터리, 뉴스, 스포츠는 일반적으로 리얼리티 쇼로 분류하지 않는다."

In [44]:
genre_df.to_csv(current+'/data/genre_df.csv', index=False)

In [ ]:

europe_genre_keywords = ["그리스", "네덜란드", "노르웨이", "덴마크", "독일", "라트비아", "루마니아", "룩셈부르크", "리투아니아", "리히텐슈타인", "모나코", "몬테네그로", "몰도바", "몰타", "벨기에", "벨라루스", "보스니아-헤르체고비나", "북마케도니아", "불가리아", "사이프러스", '산마리노', "세르비아", "스웨덴", "스위스", "스페인", "슬로바키아", "슬로베니아", "아르메니아", "아이슬란드", "아일랜드", "아제르바이잔", "안도라", "알바니아", "에스토니아", "영국", "오스트리아", "우크라이나" , "이탈리아", "조지아", "체코", "코소보", "크로아티아", "터키", "포르투갈", "폴란드", "프랑스", "핀란드", "헝가리"]

In [ ]:
save_dir = current + '/data'

query = "제주 여행"
start_date = "2020-12-01"
end_date = "2020-12-31"
max_link_cnt = dec
crawling_month = '12'

naver_df = pd.DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])

In [2]:
genre_dict = {
    '액션' : [],
    '코미디' : [],
    '다큐멘터리' : [],
    '판타지' : [],
    '공포' : [],
    '음악' : [],
    '로맨스' : [],
    '스포츠' : [],
    '서부' : [],
    'Made in Europe' : [],
    '애니메이션' : [],
    '범죄' : [],
    '드라마' : [],
    '역사' : [],
    '가족' : [],
    '스릴러' : [],
    'SF' : [],
    '전쟁' : [],
    'Reality TV' : []
}

In [5]:
genre_dict.keys()

dict_keys(['액션', '코미디', '다큐멘터리', '판타지', '공포', '음악', '로맨스', '스포츠', '서부', 'Made in Europe', '애니메이션', '범죄', '드라마', '역사', '가족', '스릴러', 'SF', '전쟁', 'Reality TV'])